In [1]:
import requests, datetime, re, time, argparse, utils
import pandas as pd

In [2]:
# 定義參數必須能夠轉換成 datetime.date 物件
def valid_date(d):
    try:
        # 透過split('-')轉換成list, 將個別的字轉換成整數, 最後將list內的整數傳給datetime.date
        return datetime.date(*[int(i) for i in d.split('-')])
    # 捕捉轉換失敗的情況
    except ValueError:
        # 定義錯誤的字串
        msg = '{} is not a valid date'.format(d)
        raise argparse.ArgumentTypeError(msg)

In [3]:
# 設定參數空間
parser = argparse.ArgumentParser()
parser.add_argument('-n', dest='n_days')
parser.add_argument('-d', '--date', help="Date - format YYYY-MM-DD ", type=valid_date)

# 在jupyter內使用, 必須透過 args=[]的方式帶入參數, 如果是再.py檔中parse_args內就不要給東西
args = parser.parse_args(args=['-n', '2', '-d', '2018-09-18'])

In [4]:
# 看到date成功設成datetime.date物件, 且n_days設定為2
args

Namespace(date=datetime.date(2018, 9, 18), n_days='2')

In [5]:
def craw_twse(date, df=None):
    # 設定header, 透過str(i).zfill()可以將整數轉成字串後補零
    header = [str(i).zfill(2) for i in ['response', 'json', 'date', date.year, date.month, date.day]]
    # 將日期的字串取出, 等等回傳使用
    date_string = '{}-{}-{}'.format(*header[-3:])
    # 將header設到url中
    url = 'http://www.twse.com.tw/exchangeReport/MI_5MINS_INDEX?{}={}&{}={}{}{}'.format(*header)
    
    while True:
        time.sleep(5)
        try:
            with requests.get(url) as response:
                # 取出json物件
                j = response.json()
                
                # 取出fields以及data放進DataFrame中
                df = pd.DataFrame(columns=j['fields'])
                for data in j['data']:
                    df.loc[data[0]] = data
                
                # 調整DataFrame成為我們想要的格式
                df.rename(columns={'時間':'Time', '發行量加權股價指數':'TWSE', '電子類指數':'TE', '金融保險類指數':'TF'}, inplace=True)
                df.drop(columns=[col for col in df.columns.values if re.search('[\u4e00-\u9fa5]', col)], inplace=True)
                df = df.set_index('Time')
                df.index.name = df.index.name=None
                
                # 印出完成字樣, 回傳日期字串及DataFrame, 記得把迴圈break
                print(url[-8:], 'Successed.')
                return date_string, df
                break
        
        # 捕捉ConnectionError
        except requests.exceptions.ConnectionError:
            print(url[-8:], 'Failed.')
            
        # 捕捉KeyError, 這會發生在上方建立 DataFrame的時候, 判斷是並沒有fields欄位, 該錯誤會在非交易日發生
        except KeyError :
            print(url[-8:], 'Not a bussiness day.')
            return date_string, None
            break

In [6]:
def main():
    # 從args中拿取變數, 沒東西就用預設從今天往回起算30天
    date = datetime.date.today() if args.date is None else args.date
    n_days = 30 if args.n_days is None else int(args.n_days)
    
    # 試著打開檔案, 失敗就直接建立空的字典
    try:
        data = utils.load_obj('twse.pkl')
    except:
        data = {}
        
    # 跑回圈抓資料, 先放進rawdata中, 再放進字典中
    days = [date - datetime.timedelta(i) for i in range(n_days)]
    rawdata = [craw_twse(day) for day in days]
    for date_string, df in rawdata:
        data[date_string] = df
    
    # 使用二進位儲存, 並印出完成字樣
    utils.save_obj(data, 'data/twse.pkl')
    print('Finished.')
    return data

In [7]:
# 執行, 看到從20180918開始, 並只有爬兩天, 完成
if __name__ == '__main__':
    data = main()

20180918 Successed.
20180917 Successed.
Finished.
